In [95]:
!pip install pydub

In [6]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
import pydub
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [7]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [8]:
#function to identify the nearest neighbors
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [9]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [12]:
directory = "C:/Users/Rahul/Desktop/IndianS"
# assert os.path.isfile(directory)

f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==6 :
        break   
    for file in os.listdir(directory+ "/" + folder):
        #         print(file)  
        (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

In [4]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  
    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)
# print(testSet)

[(array([ 73.44043494,  10.4638747 ,   2.58262276,  -0.77461022,
        13.42124638, -34.42318465,  24.62404725, -21.4471437 ,
        -4.37974943,  -0.66748224, -26.2737701 ,  10.53043427,
        -7.61508448]), array([[  82.58441717,  -58.08627679, -111.11037372,   16.23012555,
          12.50560727,  -25.9623269 ,   66.30257416,   43.40548626,
          -8.8082258 ,  116.96419486,  -96.88340581,  110.77052108,
         -54.94780538],
       [ -58.08627679,   71.84072269,   78.25870669,  -40.34795438,
          -3.42613811,    9.46470127,  -41.8578991 ,  -32.74873573,
          25.69363964,  -91.9689524 ,   76.51343071,  -92.34018303,
          60.26511083],
       [-111.11037372,   78.25870669,  195.769573  ,   17.76548164,
         -45.79371018,   53.73454341, -104.13546342,  -45.97499725,
          11.40233413, -153.92432307,  135.47114222, -140.85697701,
          72.3624948 ],
       [  16.23012555,  -40.34795438,   17.76548164,  126.78688326,
           1.89747807,   -3.653105

In [111]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [112]:
# Make the prediction using KNN(K nearest Neighbors)
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 2))) 

accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

0.8488372093023255


In [113]:
from collections import defaultdict
results = defaultdict(int)

directory = "C:/Users/Rahul/Desktop/IndianS"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [114]:
pred = nearestClass(getNeighbors(dataset,feature,5))
print(results[pred])

Sufi
